# UAS JARINGAN SYARAF TIRUAN
KELOMPOK 4:
*   Renti Epana Sari
*   Wahyu Dwi Prasetio
*   Fedryanto Dartiko
*   Aisyah Amalia Alfitri
*   Wahyu Syahputra




In [1]:
# Import TensorFlow
import tensorflow as tf

from keras import datasets
from keras import layers
from keras import models
from keras import applications
from keras import preprocessing
from keras import losses

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os

## **TRANSFER LEARNING**
Dengan menggunakan dataset yang telah digunakan sebelumnya, implementasikan Transfer Learning untuk melakukan klasifikasi

### **Data Loading**

Dataset yang digunakan ialah **Vegetable Image Dataset** yang tersedia di [Kaggle](https://www.kaggle.com/datasets/misrakahmed/vegetable-image-dataset)

Dataset terdiri atas 21000 citra yang **sudah di-split** train, test, dan validation.


In [2]:
TRAIN_DIR = '../input/vegetable-image-dataset/Vegetable Images/train'
TEST_DIR = '../input/vegetable-image-dataset/Vegetable Images/test'
VAL_DIR = '../input/vegetable-image-dataset/Vegetable Images/validation'

In [3]:
image_formats = ["png", "jpg"];

def show_images(image_files):  
    plt.figure() # tetapkan ukuran grid secara keseluruhan
    fig = plt.figure(figsize=(10,10))
    fig.patch.set_facecolor('xkcd:white')
    
    for i in range(len(image_files)):
        plt.subplot(4,5,i+1)    # jumlah gambar dalam grid yaitu 4*5 (20)
        img=mpimg.imread(image_files[i])
        plt.imshow(img)
        plt.tight_layout()
        plt.axis('off')
        plt.title(image_files[i].split("/")[5]+"\n"+"{}x{}".format(img.shape[0], img.shape[1])) # nama sayur dan ukuran gambar
    
    plt.show()

def list_files(dir):
    arr = []
    for root, dirs, files in os.walk(dir):
        for name in files:
            if name.endswith(".jpg") or name.endswith(".png"):
              arr.append(os.path.join(root, name))
              break
    return arr

Dataset terdari dari 15 kelas, yaitu bean, bitter gourd, bottle gourd, brinjal, broccoli, cabbage, capsicum, carrot, cauliflower, cucumber, papaya, potato, pumpkin, radish dan tomato. 

Gambar pada dataset ini berukuran seragam (uniform), yaitu 224*224 pixel berformat *.jpg.

In [4]:
image_list = list_files(TRAIN_DIR)
show_images(image_list)

Dataset dibagi dengan rincian sebagai berikut. 
- Data train berjumlah 15000 gambar (@1000)
- Data test berjumlah 3000 gambar (@200)
- Data validation berjumlah 3000 gambar (@200)


In [5]:
def count_files(dir):
    arr = []
    for root, dirs, files in os.walk(dir):
        count = 0
        for name in files:
            if name.endswith(".jpg") or name.endswith(".png"):
              count = count + 1
        if count > 0:
          arr.append(count)
    return arr

def get_all_veg_names(dir):
    arr = []
    for root, dirs, files in os.walk(dir):
        arr.append(dirs)
        break
    return arr

image_count = count_files(TRAIN_DIR)
print(len(image_count))
chars = get_all_veg_names(TRAIN_DIR)
# print(chars)

fig = plt.figure()
ax = fig.add_axes([0,0,3,1])
ax.bar(chars[0], image_count)
plt.title("Distribusi data train")
plt.show()

In [6]:
image_count = count_files(VAL_DIR)
print(len(image_count))
chars = get_all_veg_names(VAL_DIR)
# print(chars)

fig = plt.figure()
ax = fig.add_axes([0,0,3,1])
ax.bar(chars[0], image_count)
plt.title("Distribusi data val")
plt.show()

In [7]:
image_count = count_files(TEST_DIR)
print(len(image_count))
chars = get_all_veg_names(TEST_DIR)
# print(chars)

fig = plt.figure()
ax = fig.add_axes([0,0,3,1])
ax.bar(chars[0], image_count)
plt.title("Distribusi data test")
plt.show()

### **Preproses Data**

Melakukan preproses data menggunakan ImageDataGenerator agar data siap untuk di train. Objek datagen berfungsi untuk memproses data sebelum di load. Selain itu, penggunaan fungsi flow_from_directory akan membuat data ter-label-kan berdasarkan nama folder di mana ia tersimpan.

In [8]:
IMAGE_SIZE = (224,224)

train_datagen = preprocessing.image.ImageDataGenerator(rescale = 1./255,
                                         shear_range = 0.2,
                                         zoom_range = 0.2,
                                         horizontal_flip = True)

val_datagen = preprocessing.image.ImageDataGenerator(rescale = 1./255.)

train_generator = train_datagen.flow_from_directory(TRAIN_DIR,
                                                    batch_size = 64,
                                                    class_mode = 'categorical', 
                                                    target_size = IMAGE_SIZE) 

val_generator = val_datagen.flow_from_directory(VAL_DIR,
                                                batch_size = 64,
                                                class_mode = 'categorical', 
                                                target_size = IMAGE_SIZE) 

test_generator = val_datagen.flow_from_directory(TEST_DIR,
                                                batch_size = 64,
                                                class_mode = 'categorical', 
                                                target_size = IMAGE_SIZE) 


### **Pengembangan Model dengan Transfer Learning**

Transfer learning adalah metode yang memanfaatkan model yang sudah dilatih terhadap suatu dataset untuk menyelesaikan permasalahan lain yang serupa dengan cara menggunakannya sebagai starting point, memodifikasi dan mengupdate parameternya sehingga sesuai dengan dataset yang baru.



#### Pretrained Model InceptionV3

Pada dataset yang kami pilih, kami akan menggunakan Model InceptionV3 yang sudah dilatih dengan dataset **imageNet** untuk melakukan klasifikasi sayur-mayur.

Pretrained model InceptionV3 ini dipilih karena model telah dilatih dengan data ImageNet yang mengandung data sayuran ([Sumber](https://deeplearning.cms.waikato.ac.nz/user-guide/class-maps/IMAGENET/)), seperti *broccoli, cauliflower, zucchini, cucumber, bell pepper*, dan lainnya. Karena InceptionV3 telah memiliki weight yang mempelajari features pada sayuran, maka wajar jika kita melakukan klasifikasi sayur mayur menggunakan InceptionV3 Imagenet.

In [9]:
"""
Setting Hyperparameter
"""
EPOCHS = 5
BATCH_SIZE = 64

In [10]:
"""
Load Model InceptionV3 dengan weight dari ImageNet
"""
inception = applications.inception_v3.InceptionV3(input_shape=(224, 224, 3), weights='imagenet', include_top=False)



In [11]:
inception.summary()

#### Finetune Top/Classifier

Melatih layer output/classifier dari model InceptionV3 sehingga sesuai dengan klasifikasi sayuran yang memiliki 15 kelas. Layer lainnya yang merupakan bawaan dari InceptionV3 akan dibekukan

In [12]:
# freeze layers yang ada 
for layer in inception.layers:
    layer.trainable = False

x = inception.output

# buat layer klasifikasi yang nantinya akan dilatih
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(128,activation='relu')(x)
x = layers.Dropout(0.2)(x)

prediction = layers.Dense(15, activation='softmax')(x)

model = models.Model(inputs=inception.input, outputs=prediction)

# display the architecture of our model
model.summary()

"""
Compile Model
"""
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [13]:
"""
Training Model
"""
history = model.fit(
          train_generator,
          epochs = EPOCHS,
          steps_per_epoch = int(0.2*len(train_generator)),
          validation_data = val_generator,
          validation_steps = len(val_generator),
          verbose = 1)


In [14]:
"""
Evaluasi Model
"""
# evaluate the model accuracy
plt.plot(history.history['accuracy'], label='acc')
plt.plot(history.history['val_accuracy'], label = 'val_acc')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title("Model InceptionV3 Finetune Top Accuracy")
plt.legend(loc='lower right')

In [15]:
# evaluate the model losses
plt.plot(history.history['loss'], label='loss')
plt.plot(history.history['val_loss'], label = 'val_loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title("Model InceptionV3 Finetune Top Loss")
plt.legend(loc='upper right')

In [16]:
val_loss,val_acc = model.evaluate(val_generator)
test_loss,test_acc = model.evaluate(test_generator)

print("Val loss: {} \nVal acc: {}".format(val_loss, val_acc))
print("Test loss: {} \nTest acc: {}".format(test_loss, test_acc))

#### Finetune Backbone Layer

Melatih beberapa layer terakhir dari pretrained model InceptionV3 dengan cara di unfreeze. Untuk layer lainnya tetap di-*freeze*. Layer yang di-unfreeze ialah layer-layer setelah layer 'mixed9'

In [17]:
train_generator.reset()
val_generator.reset()
test_generator.reset()
"""
Finetune Pretrained Model InceptionV3
6 layer terakhir ikut dilatih
"""

inception2 = applications.inception_v3.InceptionV3(input_shape=(224, 224, 3), weights='imagenet', include_top=False)
# freeze layers yang ada 
for layer in inception2.layers:
    layer.trainable = False

last_layer = inception2.get_layer('mixed9')
tuning_index = inception2.layers.index(last_layer)

for layer in inception2.layers[tuning_index:]:
    layer.trainable = True
    
x2 = inception2.output

# buat layer klasifikasi yang nantinya akan dilatih
x2 = layers.GlobalAveragePooling2D()(x2)
x2 = layers.Dense(128,activation='relu')(x2)
x2 = layers.Dropout(0.2)(x2)

prediction2 = layers.Dense(15, activation='softmax')(x2)

model2 = models.Model(inputs=inception2.input, outputs=prediction2)

# display the architecture of our model
model2.summary()

"""
Compile Model
"""
model2.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [18]:
"""
Training Model
"""
history2 = model2.fit(
          train_generator,
          epochs = EPOCHS,
          steps_per_epoch = int(0.2*len(train_generator)),
          validation_data = val_generator,
          validation_steps = len(val_generator),
          verbose = 1)


In [19]:
"""
Evaluasi Model
"""
# evaluate the model accuracy
plt.plot(history2.history['accuracy'], label='acc')
plt.plot(history2.history['val_accuracy'], label = 'val_acc')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title("Model InceptionV3 Finetune Backbone Accuracy")
plt.legend(loc='lower right')

In [20]:
# evaluate the model losses
plt.plot(history2.history['loss'], label='loss')
plt.plot(history2.history['val_loss'], label = 'val_loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title("Model InceptionV3 Finetune Backbone Loss")
plt.legend(loc='upper right')

In [21]:
val_loss,val_acc = model2.evaluate(val_generator)
test_loss,test_acc = model2.evaluate(test_generator)

print("Val loss: {} \nVal acc: {}".format(val_loss, val_acc))
print("Test loss: {} \nTest acc: {}".format(test_loss, test_acc))

#### **Kesimpulan**: dengan menggunakan pretrained model InceptionV3 yang dilatih dengan data ImageNet, kami berhasil melakukan klasifikasi terhadap dataset sayur-mayur. Nilai akurasi yang didapat, diantaranya
- Untuk model dengan finetune classifier, akurasi validationnya 0.989 dan akurasi testnya 0.986
- Untuk model dengan finetune backbone, di mana layer setelah 'mixed9' di unfreeze, akurasi validationnya 0.994 dan akurasi testnya 0.992

